In [1]:
from glob import glob
import string
import numpy as np
import tensorflow as tf
from datetime import datetime
from scipy.special import expit as sigmoid

import os
import sys
import string
import json


from nltk.corpus import brown
import operator

from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import cosine as cos_dist


In [2]:
def remove_punctuation(s):
  return s.translate(str.maketrans('','',string.punctuation))

In [3]:
def get_wiki():
  V = 20000
  files = glob('../large_files/enwiki*.txt')
  all_word_counts = {}
  for f in files:
    for line in open(f, encoding="utf8"):
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:
          for word in s:
            if word not in all_word_counts:
              all_word_counts[word] = 0
            all_word_counts[word] += 1
  print("finished counting")

  V = min(V, len(all_word_counts))
  all_word_counts = sorted(all_word_counts.items(), key=lambda x: x[1], reverse=True)

  top_words = [w for w, count in all_word_counts[:V-1]] + ['<UNK>']
  word2idx = {w:i for i, w in enumerate(top_words)}
  unk = word2idx['<UNK>']

  sents = []
  for f in files:
    for line in open(f, encoding="utf8"):
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:
          # if a word is not nearby another word, there won't be any context!
          # and hence nothing to train!
          sent = [word2idx[w] if w in word2idx else unk for w in s]
          sents.append(sent)
  return sents, word2idx
              

In [4]:
def get_sentences():
  return brown.sents()

def get_brown(n_vocab=2000, keep_words = []):
  sentences = get_sentences()
  indexed_sentences = []

  i = 0
  word2idx = {}
  idx2word = []

  word_idx_count = {}

  for sentence in sentences:
    indexed_sentence = []
    for token in sentence:
      token = token.lower()
      if token not in word2idx:
        idx2word.append(token)
        word2idx[token] = i
        i += 1

      idx = word2idx[token]
      word_idx_count[idx] = word_idx_count.get(idx, 0) + 1

      indexed_sentence.append(idx)
    indexed_sentences.append(indexed_sentence)

  for word in keep_words:
    word_idx_count[word2idx[word]] = float('inf')

  sorted_word_idx_count = sorted(word_idx_count.items(), key=operator.itemgetter(1), reverse=True)
  word2idx_small = {}
  new_idx = 0
  idx_new_idx_map = {}
  for idx, count in sorted_word_idx_count[:n_vocab]:
    word = idx2word[idx]
    word2idx_small[word] = new_idx
    idx_new_idx_map[idx] = new_idx
    new_idx += 1
  word2idx_small['UNKNOWN'] = new_idx 
  unknown = new_idx

  sentences_small = []
  for sentence in indexed_sentences:
    if len(sentence) > 1:
      new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
      sentences_small.append(new_sentence)

  return sentences_small, word2idx_small


In [5]:
def construct_matrix(cc_matrix, sentences, V, context_sz):
  if not os.path.exists(cc_matrix):
      X = np.zeros((V, V))
      N = len(sentences)
      print("number of sentences to process:", N)
      it = 0
      for sentence in sentences:
          it += 1
          if it % 10000 == 0:
              print("processed", it, "/", N)
          n = len(sentence)
          for i in range(n):
              wi = sentence[i]

              start = max(0, i - context_sz)
              end = min(n, i + context_sz)

              if i - context_sz < 0:
                  points = 1.0 / (i + 1)
                  X[wi,0] += points
                  X[0,wi] += points
              if i + context_sz > n:
                  points = 1.0 / (n - i)
                  X[wi,1] += points
                  X[1,wi] += points

              for j in range(start, i):
                  wj = sentence[j]
                  points = 1.0 / (i - j) # this is +ve
                  X[wi,wj] += points
                  X[wj,wi] += points

              for j in range(i + 1, end):
                  wj = sentence[j]
                  points = 1.0 / (j - i) # this is +ve
                  X[wi,wj] += points
                  X[wj,wi] += points

      np.save(cc_matrix, X)

In [6]:
def train(cc_matrix, D, learning_rate=1e-5, reg=0.1, xmax=100, alpha=0.75, epochs=100):
  tf.compat.v1.disable_eager_execution()
  X = np.load(cc_matrix)
  V = len(X)
  print("max in X:", X.max())

  # weighting
  fX = np.zeros((V, V))
  fX[X < xmax] = (X[X < xmax] / float(xmax)) ** alpha
  fX[X >= xmax] = 1

  print("max in f(X):", fX.max())

  # target
  logX = np.log(X + 1)

  print("max in log(X):", logX.max())

  # initialize weights
  W = np.random.randn(V, D) / np.sqrt(V + D)
  b = np.zeros(V)
  U = np.random.randn(V, D) / np.sqrt(V + D)
  c = np.zeros(V)
  mu = logX.mean()

  # initialize weights, inputs, targets placeholders
  tfW = tf.Variable(W.astype(np.float32))
  tfb = tf.Variable(b.reshape(V, 1).astype(np.float32))
  tfU = tf.Variable(U.astype(np.float32))
  tfc = tf.Variable(c.reshape(1, V).astype(np.float32))
  tfLogX = tf.compat.v1.placeholder(tf.float32, shape=(V, V))
  tffX = tf.compat.v1.placeholder(tf.float32, shape=(V, V))

  delta = tf.matmul(tfW, tf.transpose(a=tfU)) + tfb + tfc + mu - tfLogX
  cost = tf.reduce_sum(input_tensor=tffX * delta * delta)
  regularized_cost = cost
  for param in (tfW, tfU):
      regularized_cost += reg*tf.reduce_sum(input_tensor=param * param)

  train_op = tf.compat.v1.train.MomentumOptimizer(
    learning_rate,
    momentum=0.9
  ).minimize(regularized_cost)
  # train_op = tf.train.AdamOptimizer(1e-3).minimize(regularized_cost)
  init = tf.compat.v1.global_variables_initializer()
  session = tf.compat.v1.InteractiveSession()
  session.run(init)

  costs = []
  for epoch in range(epochs):
      c, _ = session.run((cost, train_op), feed_dict={tfLogX: logX, tffX: fX})
      print("epoch:", epoch, "cost:", c)
      costs.append(c)

  # save for future calculations
  W, U = session.run([tfW, tfU])
  return W, U

In [11]:
sentences, word2idx = get_wiki()


finished counting


In [8]:

# construct_matrix('cc_matrix_wiki', sentences, V, 10)
W, U = train('cc_matrix_wiki.npy', 100)

max in X: 10554944.031889495
max in f(X): 1.0
max in log(X): 16.1721050314717
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
epoch: 0 cost: 48124384.0
epoch: 1 cost: 35185476.0
epoch: 2 cost: 22557570.0
epoch: 3 cost: 18661240.0
epoch: 4 cost: 19930980.0
epoch: 5 cost: 19206512.0
epoch: 6 cost: 15427730.0
epoch: 7 cost: 12667851.0
epoch: 8 cost: 12987077.0
epoch: 9 cost: 14055838.0
epoch: 10 cost: 13322813.0
epoch: 11 cost: 11453230.0
epoch: 12 cost: 10582043.0
epoch: 13 cost: 10983856.0
epoch: 14 cost: 11075684.0
epoch: 15 cost: 10021385.0
epoch: 16 cost: 8784889.0
epoch: 17 cost: 8465030.0
epoch: 18 cost: 8736060.0
epoch: 19 cost: 8544645.0
epoch: 20 cost: 7702225.0
epoch: 21 cost: 7014430.0
epoch: 22 cost: 7011961.0
epoch: 23 cost: 7256971.0
epoch: 24 cost: 7080768.0
epoch: 25 cost: 6518613.5
epoch: 26 cost: 6147890.5
epoch: 27 cost: 6228239.0
epoch: 28 cost: 6404731.0
epoch: 29 cost: 6267929.0
epoch: 30 cost: 5887027.0
epoch: 31 cost: 5633744.5
epo

In [9]:
def find_analogies(w1, w2, w3, We, word2idx, idx2word):
    V, D = We.shape

    king = We[word2idx[w1]]
    man = We[word2idx[w2]]
    woman = We[word2idx[w3]]
    v0 = king - man + woman

    for dist in ('euclidean', 'cosine'):
        distances = pairwise_distances(v0.reshape(1, D), We, metric=dist).reshape(V)
        # idx = distances.argmin()
        # best_word = idx2word[idx]
        idx = distances.argsort()[:4]
        best_idx = -1
        keep_out = [word2idx[w] for w in (w1, w2, w3)]
        for i in idx:
            if i not in keep_out:
                best_idx = i
                break
        best_word = idx2word[best_idx]


        print("closest match by", dist, "distance:", best_word)
        print(w1, "-", w2, "=", best_word, "-", w3)

In [12]:
print(W.shape, U.shape)
idx2word = {i:w for w, i in word2idx.items()}
We = (W + U) / 2

find_analogies('king', 'man', 'woman', We, word2idx, idx2word)
find_analogies('france', 'paris', 'london', We, word2idx, idx2word)
find_analogies('france', 'paris', 'rome', We, word2idx, idx2word)
find_analogies('paris', 'france', 'italy', We, word2idx, idx2word)
find_analogies('france', 'french', 'english', We, word2idx, idx2word)
find_analogies('japan', 'japanese', 'chinese', We, word2idx, idx2word)
find_analogies('japan', 'japanese', 'italian', We, word2idx, idx2word)
find_analogies('japan', 'japanese', 'australian', We, word2idx, idx2word)
find_analogies('december', 'november', 'june', We, word2idx, idx2word)

(20000, 100) (20000, 100)
closest match by euclidean distance: charles
king - man = charles - woman
closest match by cosine distance: ii
king - man = ii - woman
closest match by euclidean distance: europe
france - paris = europe - london
closest match by cosine distance: united
france - paris = united - london
closest match by euclidean distance: england
france - paris = england - rome
closest match by cosine distance: war
france - paris = war - rome
closest match by euclidean distance: ceased
paris - france = ceased - italy
closest match by cosine distance: runs
paris - france = runs - italy
closest match by euclidean distance: america
france - french = america - english
closest match by cosine distance: europe
france - french = europe - english
closest match by euclidean distance: australia
japan - japanese = australia - chinese
closest match by cosine distance: south
japan - japanese = south - chinese
closest match by euclidean distance: decline
japan - japanese = decline - italian
